In [8]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split

# 데이터 로드
with open('audio_data.pkl', 'rb') as f:
    data = pickle.load(f)

# 데이터프레임 생성
df = pd.DataFrame(data)

# 데이터를 8:1:1 비율로 나누기
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

# 데이터 개수 확인
print(f"전체 데이터 개수: {len(df)}")
print(f"언어 데이터 개수: {len(df[df['label'] == 0])}")
print(f"청각 데이터 개수: {len(df[df['label'] == 1])}")

print(f"Train 데이터 개수: {len(train_df)}")
print(f"Validation 데이터 개수: {len(valid_df)}")
print(f"Test 데이터 개수: {len(test_df)}")

# 각 데이터셋의 레이블 비율 확인
def print_label_distribution(dataset, dataset_name):
    label_counts = dataset['label'].value_counts()
    print(f"\n{dataset_name} 데이터 레이블 분포:")
    for label, count in label_counts.items():
        print(f"라벨 {label}: {count} ({count / len(dataset) * 100:.2f}%)")

print_label_distribution(df, "전체")
print_label_distribution(train_df, "Train")
print_label_distribution(valid_df, "Validation")
print_label_distribution(test_df, "Test")

SyntaxError: invalid syntax (<unknown>, line 4)

In [ ]:
# 데이터와 레이블 준비
X_train = np.array([x for x in train_df['log_mel_spec']])
y_train = np.array([y for y in train_df['label']])

X_valid = np.array([x for x in valid_df['log_mel_spec']])
y_valid = np.array([y for y in valid_df['label']])

X_test = np.array([x for x in test_df['log_mel_spec']])
y_test = np.array([y for y in test_df['label']])

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, AveragePooling2D, Flatten, Dense
# 모델 구성
model = Sequential([
    Conv2D(8, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 3)),
    BatchNormalization(),
    AveragePooling2D(pool_size=(2, 2)),

    Conv2D(16, kernel_size=(3, 3), activation='relu'),
    BatchNormalization(),
    AveragePooling2D(pool_size=(2, 2)),

    Conv2D(32, kernel_size=(3, 3), activation='relu'),
    BatchNormalization(),
    AveragePooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(512, activation='relu'),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(2, activation='softmax')
    ])# 클래스 수에 따라 출력 노드 수 조정

In [ ]:
# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_valid, y_valid))

# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")